In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

genderclassmodel.csv
gendermodel.csv
gendermodel.py
myfirstforest.py
test.csv
train.csv



In [2]:
# Load the data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.groupby('Sex').Survived.value_counts()

Sex     Survived
female  1           233
        0            81
male    0           468
        1           109
Name: Survived, dtype: int64

In [4]:
train.groupby(['Pclass','Sex']).Survived.value_counts()

Pclass  Sex     Survived
1       female  1            91
                0             3
        male    0            77
                1            45
2       female  1            70
                0             6
        male    0            91
                1            17
3       female  0            72
                1            72
        male    0           300
                1            47
Name: Survived, dtype: int64

In [5]:
id = pd.crosstab([train.Pclass,train.Sex],train.Survived.astype(float))
id.div(id.sum(1).astype(float),0)

Survived            0.0       1.0
Pclass Sex                       
1      female  0.031915  0.968085
       male    0.631148  0.368852
2      female  0.078947  0.921053
       male    0.842593  0.157407
3      female  0.500000  0.500000
       male    0.864553  0.135447

In [6]:
#all the data to be in numerical format. As we can see below, our data set has 5 categorical variables which contain non-numerical values: Name, Sex, Ticket, Cabin and Embarked.
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
#We then check the number of levels that each of the five categorical variables have.
for cat in ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']:
    print("Number of levels in category'{0}' : {1:2.2f}".format(cat,train[cat].unique().size))

Number of levels in category'Name' : 891.00
Number of levels in category'Sex' : 2.00
Number of levels in category'Ticket' : 681.00
Number of levels in category'Cabin' : 148.00
Number of levels in category'Embarked' : 4.00


In [8]:
#explor Sex & Embarked
for cat in ['Sex', 'Embarked']:
    print ('category  :{} - Values : {}'.format(cat,train[cat].unique()))

category  :Sex - Values : ['male' 'female']
category  :Embarked - Values : ['S' 'C' 'Q' nan]


In [9]:
#map Sex & Embarked
train['Sex'] = train['Sex'].map({'male':0,'female':1})
train['Embarked']=train['Embarked'].map({'S':0,'C':1,'Q':2})
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0.0


In [10]:
#For nan i.e. the missing values, we simply replace them with a placeholder value (-999). In fact, we perform this replacement for the entire data set
train=train.fillna(-999)
pd.isnull(train).any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

In [11]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,-999,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,-999,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,-999,0.0


In [12]:
#For Cabin, we encode the levels as digits using Scikit-learn's MultiLabelBinarizer and treat them as new features.
from sklearn.preprocessing import MultiLabelBinarizer
mlb= MultiLabelBinarizer()
CabinTrans = mlb.fit_transform([{str(val)} for val in train['Cabin'].values ])
CabinTrans

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [13]:
#Name and Ticket have so many levels , drop 
#Survives drop target 
#Cabin drop - set new feature
train_new = train.drop(['Name','Ticket','Cabin','Survived'],axis=1)

In [14]:
#check correct encoding done
assert (len(train['Cabin'].unique()) == len(mlb.classes_)),"not equal"

In [15]:
train_new= np.hstack((train_new.values,CabinTrans))

In [16]:
print (np.isnan(train_new).any())
print (train_new[0].size)

False
156


In [17]:
#store the Survived labels, which we need to predict, in a separate variable.
train_class=train['Survived'].values

In [18]:
from sklearn.model_selection import train_test_split
training_features, testing_features, training_classes, testing_classes = train_test_split(train_new, train_class, train_size=0.75, test_size=0.25)

In [19]:
from sklearn.ensemble import RandomForestClassifier
export_pipline = RandomForestClassifier(bootstrap=False,max_features=0.4,min_samples_leaf=1,min_samples_split=9)


In [20]:
export_pipline.fit(training_features,training_classes)
result = export_pipline.predict(testing_features)
result

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0])

In [21]:
s_prob = pd.DataFrame(export_pipline.predict_proba(testing_features),columns=['P({})'.format(x) for x in export_pipline.classes_])
s_prob.head()


,P(0),P(1)
0,0.966667,0.033333
1,0.519643,0.480357
2,0.900000,0.100000
3,0.683929,0.316071
4,0.000000,1.000000


In [22]:
print(export_pipline.score(train_new,train_class))

0.933782267116


In [23]:
export_pipline.classes_

array([0, 1])

In [24]:
s_prob= pd.DataFrame(export_pipline.predict_proba(training_features),columns=['P({})'.format(x) for x in export_pipline.classes_] )
s_prob

,P(0),P(1)
0,0.397024,0.602976
1,0.983333,0.016667
2,0.819643,0.180357
3,0.888929,0.111071
4,1.000000,0.000000
5,0.931429,0.068571
6,0.077500,0.922500
7,0.902381,0.097619
8,0.215833,0.784167
9,0.125000,0.875000


In [25]:
#prediction on test data
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [26]:
test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [27]:
for cat in ['Cabin','Name','Sex','Embarked','Ticket']:
    print ("unique values in {} : {}".format(cat,test[cat].unique().size))

unique values in Cabin : 77
unique values in Name : 418
unique values in Sex : 2
unique values in Embarked : 3
unique values in Ticket : 363


In [28]:
for var in ['Cabin','Name','Ticket']:
    new = list(set(test[var])- set(train[var]))
    test.ix[test[var].isin(new) ,var] =-999

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
test['Sex'] =test['Sex'].map({'male':0,'female':1})
test['Embarked'] = test['Embarked'].map({'S':0,'C':1,'Q':2})

In [30]:
test=test.fillna(-999)
pd.isnull(test).any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb= MultiLabelBinarizer()
subCabinTrans=mlb.fit([{str(val)} for val in train['Cabin'].values]).transform([{str(val)} for val in test['Cabin'].values ])

In [32]:
test = test.drop(['Name','Ticket','Cabin'], axis=1)

In [33]:

# Form the new submission data set
test_new = np.hstack((test.values,subCabinTrans))

In [34]:
# Ensure equal number of features in both the final training and submission dataset
assert (train_new.shape[1] == train_new.shape[1]), "Not Equal"

In [35]:
submission  = export_pipline.predict(test_new)
submission

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0,

In [36]:
# Create the submission file
result = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived': submission})
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [37]:
result.to_csv('submission.csv',index=False)